In [13]:
%cd tailscale_all

/home/griffin/git/griffinht.com/notebooks/tailscale_all


In [ ]:
Prereqs:
You will need a Kubernetes
I tried this using Kind and it did not work!
Minikube worked
Haven't tried with k3d yet.

In [31]:
%%writefile pod.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: tailscale-state
spec:
  accessModes:
    # read/write from a single node
    # https://kubernetes.io/docs/concepts/storage/persistent-volumes/#access-modes
    - ReadWriteOnce
  # todo idk where the docs are for this - i just guessed
  resources:
    requests:
      storage: 10Mi
---
apiVersion: v1
kind: Pod
metadata:
    name: caddy
spec:
    containers:
      - name: iperf3
        image: networkstatic/iperf3
        args:
          - --server
      - name: caddy
        image: caddy/caddy
      - name: tailscale
        image: ghcr.io/tailscale/tailscale:latest
        env:
          # disable Kubernetes secrets and use OAuth login instead
          - name: TS_KUBE_SECRET
            value: ""
          - name: TS_STATE_DIR
            value: "/var/lib/tailscale"
        volumeMounts:
          - name: tailscale-state
            mountPath: "/var/lib/tailscale"
    volumes:
      - name: tailscale-state
        persistentVolumeClaim:
          claimName: tailscale-state


Overwriting pod.yaml


In [34]:
%%sh
kubectl apply -f pod.yaml

persistentvolumeclaim/tailscale-state created
pod/caddy created


Tip: If your pod is taking a while to come up, check out `kubectl describe`

In [24]:
%%sh
kubectl describe pod caddy

Name:             caddy
Namespace:        default
Priority:         0
Service Account:  default
Node:             gk3-my-cluster-pool-3-31f1207c-nfrt/10.142.0.7
Start Time:       Sun, 23 Jun 2024 18:25:21 -0400
Labels:           <none>
Annotations:      autopilot.gke.io/resource-adjustment:
                    {"input":{"containers":[{"name":"iperf3"},{"name":"caddy"},{"name":"tailscale"}]},"output":{"containers":[{"limits":{"cpu":"500m","ephemer...
                  autopilot.gke.io/warden-version: 2.9.37
Status:           Running
SeccompProfile:   RuntimeDefault
IP:               10.15.0.133
IPs:
  IP:  10.15.0.133
Containers:
  iperf3:
    Container ID:   containerd://b2bc732be4b3a938eea50dae2bb9e5dbdf8e14099c6fcf5d9a8c94dcaa501dd6
    Image:          networkstatic/iperf3
    Image ID:       docker.io/networkstatic/iperf3@sha256:148ec0389185caa366802a996ccded1bcde221a257f3eb44f291a7ac765f4ed4
    Port:           <none>
    Host Port:      <none>
    State:          Waiting
      Rea

In [28]:
%%sh
iperf3 -c caddy

Connecting to host caddy, port 5201
[  5] local 100.99.123.66 port 39614 connected to 100.123.205.111 port 5201
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-1.00   sec  11.8 MBytes  98.5 Mbits/sec  1608    186 KBytes       
[  5]   1.00-2.00   sec  8.12 MBytes  68.2 Mbits/sec    0    211 KBytes       
[  5]   2.00-3.00   sec  8.25 MBytes  69.2 Mbits/sec    0    239 KBytes       
[  5]   3.00-4.00   sec  9.62 MBytes  80.7 Mbits/sec    5    199 KBytes       
[  5]   4.00-5.00   sec  6.88 MBytes  57.7 Mbits/sec   11    159 KBytes       
[  5]   5.00-6.00   sec  8.25 MBytes  69.2 Mbits/sec    0    187 KBytes       
[  5]   6.00-7.00   sec  8.12 MBytes  68.2 Mbits/sec    0    216 KBytes       
[  5]   7.00-8.00   sec  9.62 MBytes  80.7 Mbits/sec   25    174 KBytes       
[  5]   8.00-9.00   sec  8.12 MBytes  68.2 Mbits/sec    0    204 KBytes       
[  5]   9.00-10.00  sec  8.25 MBytes  69.2 Mbits/sec    0    227 KBytes       
- - - - - - - - - - - - - - - - 

It works! We added `iperf3` to our pod, and connected to it via Tailscale.

I want to separate the two.

In [10]:
%%writefile pod.yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: tailscale-state
spec:
  accessModes:
    # read/write from a single node
    # https://kubernetes.io/docs/concepts/storage/persistent-volumes/#access-modes
    - ReadWriteOnce
  # todo idk where the docs are for this - i just guessed
  resources:
    requests:
      storage: 10Mi
---
apiVersion: v1
kind: Pod
metadata:
    name: iperf3
spec:
    containers:
      - name: iperf3
        image: networkstatic/iperf3
        args:
          - --server
---
apiVersion: v1
kind: Pod
metadata:
    name: caddy
spec:
    containers:
      - name: caddy
        image: caddy/caddy
---
apiVersion: v1
kind: Pod
metadata:
    name: tailscale
spec:
    containers:
      - name: tailscale
        image: ghcr.io/tailscale/tailscale:latest
        env:
          # disable Kubernetes secrets and use OAuth login instead
          - name: TS_KUBE_SECRET
            value: ""
          - name: TS_STATE_DIR
            value: "/var/lib/tailscale"
        volumeMounts:
          - name: tailscale-state
            mountPath: "/var/lib/tailscale"
    volumes:
      - name: tailscale-state
        persistentVolumeClaim:
          claimName: tailscale-state


Overwriting pod.yaml


In [11]:
! kubectl apply -f pod.yaml

persistentvolumeclaim/tailscale-state unchanged
pod/iperf3 unchanged
pod/caddy unchanged
pod/tailscale configured


In [8]:
! tailscale status | grep tailscale

100.113.140.69  tailscale            griffinht@   linux   -


Great! Now Tailscale shows up in its own pod as the "tailscale" hostname. We could also change the hostname if we wanted to, but I'll leave it like this for now.

In [9]:
! curl tailscale:80

curl: (7) Failed to connect to tailscale port 80 after 95 ms: Couldn't connect to server


As we saw in the Podman pod lab todo. Caddy and Iperf3 are no longer in the same pod as Tailscale, and can't be reached via localhost. Let's try to reach Caddy from Tailscale, like we would in Docker Compose. todo docker compose lab

In [27]:
! kubectl exec tailscale -- wget caddy:80

wget: bad address 'caddy:80'
command terminated with exit code 1


In [29]:
! kubectl get pods -o wide

NAME        READY   STATUS    RESTARTS   AGE   IP            NODE                 NOMINATED NODE   READINESS GATES
caddy       1/1     Running   0          13h   10.244.0.11   kind-control-plane   <none>           <none>
iperf3      1/1     Running   0          13h   10.244.0.10   kind-control-plane   <none>           <none>
tailscale   1/1     Running   0          27m   10.244.0.14   kind-control-plane   <none>           <none>


In [32]:
! kubectl exec tailscale -- wget 10.244.0.11:80

Connecting to 10.244.0.11:80 (10.244.0.11:80)
saving to 'index.html'
index.html           100% |********************************| 12256  0:00:00 ETA
'index.html' saved


Found it! Caddy is still accessible from the Tailscale pod, but doesn't seem to have any DNS name it would if we were in Docker land. todo link to docker lab?

Isolation? What if I wanted to isolate my pods? todo https://kubernetes.io/docs/concepts/services-networking/network-policies/

# Introducing Services

todo tailscale cast

In [50]:
%%writefile pod.yaml
apiVersion: v1
kind: Pod
metadata:
    name: iperf3
    labels:
      app: iperf3
spec:
    containers:
      - name: iperf3
        image: networkstatic/iperf3
        args:
          - --server
---
apiVersion: v1
kind: Pod
metadata:
    name: caddy
    labels:
      app: caddy
spec:
    containers:
      - name: caddy
        image: caddy/caddy
---
apiVersion: v1
kind: Service
metadata:
    name: caddy
spec:
    selector:
        app: caddy
    ports:
      - protocol: TCP
        port: 80
---
apiVersion: v1
kind: Service
metadata:
    name: iperf3
spec:
    selector:
        app: iperf3
    ports:
      - protocol: TCP
        port: 5201
        # todo does iperf also need UDP?
---

Overwriting pod.yaml


I just added the labels `app: caddy` and `app: iperf3` to the respective pods. In the Service, I am using those labels to target my apps.

Think this looks complex? Think again! Check out my lab on container-container networking in Docker Compose. I haven't created it yet but I want to! This [Service](https://kubernetes.io/docs/concepts/services-networking/service/) definition may seem complex, but so is todo.

Think of a service as an exposed port in Docker. The ser actually not really

This medium article on [Mastering Kubernetes Pod-to-Pod Communication](https://medium.com/@extio/mastering-kubernetes-pod-to-pod-communication-a-comprehensive-guide-46832b30556b) was also great.

In [51]:
! kubectl apply -f pod.yaml

pod/iperf3 unchanged
pod/caddy unchanged
service/caddy created
service/iperf3 created


In [44]:
! kubectl exec tailscale -- wget caddy-service:80

Connecting to caddy-service:80 (10.96.43.142:80)
saving to 'index.html'
index.html           100% |********************************| 12256  0:00:00 ETA
'index.html' saved


In [45]:
! curl tailscale:80

curl: (7) Failed to connect to tailscale port 80 after 488 ms: Couldn't connect to server


It works! But as expected, we still can't reach Caddy by connecting directly to Tailscale.

Just like in the Docker lab, there are two solutions:
The first is to configure Tailscale act as a reverse proxy to Caddy. Since we can reach Tailscale and Tailscale can reach Caddy, we can have Tailscale reverse proxy our HTTP requests to Caddy for us.

However, this would only work for one service at a time, and at that point we might as well leave Tailscale in the Caddy pod. I want to be able to access iperf3 and Caddy. I could run two Tailscales in each pod, but I'd rather have one Tailscale instance act as a VPN for me. 

In [54]:
! kubectl get services caddy iperf3

NAME     TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
caddy    ClusterIP   10.96.103.205   <none>        80/TCP     35s
iperf3   ClusterIP   10.96.141.159   <none>        5201/TCP   34s


In [125]:
%%writefile pod.yaml
apiVersion: v1
kind: Pod
metadata:
    name: tailscale
spec:
    containers:
      - name: tailscale
        image: ghcr.io/tailscale/tailscale:latest
        env:
          # disable Kubernetes secrets and use OAuth login instead
          - name: TS_KUBE_SECRET
            value: ""
          - name: TS_USERSPACE
            value: "false"
          - name: TS_DEST_IP
            value: "10.96.0.0/16"
        volumeMounts:
          - name: tun-device
            mountPath: /dev/net/tun
        securityContext:
          capabilities:
            add:
              - NET_ADMIN
    volumes:
      - name: tun-device
        hostPath:
          path: /dev/net/tun

Overwriting pod.yaml


In [126]:
! kubectl apply --force -f pod.yaml

pod/tailscale configured


In [124]:
! kubectl logs tailscale

boot: 2024/06/24 13:31:56 Failed to enable IP forwarding: invalid cluster destination IP: ParseAddr("10.96.0.0/16"): unexpected character (at "/16")
boot: 2024/06/24 13:31:56 To run tailscale as a proxy or router container, IP forwarding must be enabled.
boot: 2024/06/24 13:31:56 You can either set the sysctls as a privileged initContainer, or run the tailscale container with privileged=true.


In [107]:
! nix profile install nixpkgs#k3d

32;1m1 copied (18.0 MiB), 5.0 MiB DL]m/18.0 MiB), 5.0 MiB DL] fetching k3d-5.6.0 from ht

In [106]:
! minikube start

😄  minikube v1.33.1 on Debian 12.5
    ▪ MINIKUBE_WANTUPDATENOTIFICATION=false
✨  Automatically selected the docker driver
📌  Using rootless Docker driver
👍  Starting "minikube" primary control-plane node in "minikube" cluster
🚜  Pulling base image v0.0.44 ...
💾  Downloading Kubernetes v1.30.0 preload ...
    > preloaded-images-k8s-v18-v1...:  342.90 MiB / 342.90 MiB  100.00% 19.89 M
    > gcr.io/k8s-minikube/kicbase...:  481.58 MiB / 481.58 MiB  100.00% 16.20 M
🔥  Creating docker container (CPUs=2, Memory=3400MB) ...
🐳  Preparing Kubernetes v1.30.0 on Docker 26.1.1 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔗  Configuring bridge CNI (Container Networking Interface) ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by d

In [111]:
! kubectl config current-context

minikube
